<a href="https://colab.research.google.com/github/SalimuKabogere/XAI/blob/main/Tresnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install tensorflow keras numpy matplotlib

In [5]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from google.colab import files
from sklearn.utils.class_weight import compute_class_weight

In [6]:
from google.colab import drive  # <-- Import drive
drive.mount('/content/drive') # Mount to the default path without spaces

# Now access your dataset using the full path:
train_dir = "/content/drive/MyDrive/Dataset4/Training"
validation_dir = "/content/drive/MyDrive/Dataset4/Validation"
test_dir = "/content/drive/MyDrive/Dataset4/Testing"  # If needed for testing

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Image dimensions
height = 300
width = 300
batch_size = 8


In [8]:
# Load ResNet50 without top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(height, width, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [9]:
# Data

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,  # Add shear
    zoom_range=0.2,   # Add zoom
    brightness_range=[0.8, 1.2]  # Add brightness variation
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
 # Added test data generator

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)
class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights = dict(enumerate(class_weights))
validation_generator = val_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 64 images belonging to 2 classes.
Found 64 images belonging to 2 classes.
Found 6 images belonging to 3 classes.


In [10]:
# Add custom layers to ResNet50
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)

In [11]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(
    train_generator,
    validation_data=train_generator,
    epochs=5,
    # callbacks=[checkpoint, early_stopping],
    class_weight=class_weights  # Add class weights
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 194s 17s/step - accuracy: 0.4598 - loss: 49.3833 - val_accuracy: 0.4844 - val_loss: 5122.0845
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 135s 17s/step - accuracy: 0.4809 - loss: 15.3717 - val_accuracy: 0.5000 - val_loss: 622395.0625
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 147s 19s/step - accuracy: 0.4929 - loss: 7.4882 - val_accuracy: 0.5469 - val_loss: 18627294.0000
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 146s 19s/step - accuracy: 0.5861 - loss: 5.9107 - val_accuracy: 0.5938 - val_loss: 144069120.0000
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 127s 16s/step - accuracy: 0.6932 - loss: 10.7864 - val_accuracy: 0.5312 - val_loss: 9486230.0000


In [14]:
# Save the final model to your Google Drive
model.save("/content/drive/MyDrive/resnet50_model.h5") # or another suitable path within your Drive
print("Model training complete and saved as /content/drive/MyDrive/resnet50_model.h5")

Model training complete and saved as /content/drive/MyDrive/resnet50_model.h5
